In [3]:
'''
Text Generation Project
We will use data from project gutenberg
Book used here is Pride and Prejudice by Jane Austen

'''

'\nText Generation Project\nWe will use data from project gutenberg\nBook used here is Pride and Prejudice by Jane Austen\n\n'

In [4]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
import numpy as np

In [6]:
book= open('C:\\Users\\Dell\\Desktop\\ML\\Projects\\Text-Generation\\pg1342.txt','r',encoding='utf-8').read()
book=book.lower()

In [7]:
sentences=book.split('\n')

In [8]:
tokenizer=Tokenizer(oov_token='<UNK>')

In [9]:
tokenizer.fit_on_texts(sentences)
vocab_size= len(tokenizer.word_index)+1
vocab_size

7547

In [10]:
sequences=tokenizer.texts_to_sequences(sentences)

in_sequences=[]
for sequence in sequences:
    for i in range(1,len(sequence)):
        n_gram_sequence=sequence[:i+1]
        in_sequences.append(n_gram_sequence)

In [11]:
max_seq_len= max([len(seq) for seq in in_sequences])
max_seq_len

19

In [12]:
padded_seq = pad_sequences(in_sequences,maxlen=max_seq_len)
padded_seq[30]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   96,
         81, 1454,    4,    2,  219,   27,   44, 1614])

In [13]:
padded_seq=np.array(padded_seq)
x=padded_seq[:,:-1]
labels=padded_seq[:,-1]

In [14]:
y=tf.keras.utils.to_categorical(labels, num_classes=vocab_size)
y.shape

(121158, 7547)

In [15]:
x.shape

(121158, 18)

In [16]:
#import dependencies for training the model
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Bidirectional, Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

In [17]:
# define the model 


model=Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size, activation='softmax'))
adam= Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 100)           754700    
                                                                 
 bidirectional (Bidirectiona  (None, 512)              731136    
 l)                                                              
                                                                 
 dense (Dense)               (None, 7547)              3871611   
                                                                 
Total params: 5,357,447
Trainable params: 5,357,447
Non-trainable params: 0
_________________________________________________________________


In [18]:
#if there is not even 1% improvement in the accuracy, call early stopping 
from tensorflow.keras.callbacks import EarlyStopping 
es=EarlyStopping(monitor='acc',min_delta=0.001)

In [19]:
#fitting the model

model.fit(x,y,epochs=10, verbose=1,batch_size=512)

Epoch 1/10
237/237 [==============================] - 314s 1s/step - loss: 6.0212 - acc: 0.0860
Epoch 2/10
237/237 [==============================] - 330s 1s/step - loss: 5.1545 - acc: 0.1444
Epoch 3/10
237/237 [==============================] - 305s 1s/step - loss: 4.7087 - acc: 0.1695
Epoch 4/10
237/237 [==============================] - 302s 1s/step - loss: 4.3116 - acc: 0.1922
Epoch 5/10
237/237 [==============================] - 311s 1s/step - loss: 3.9285 - acc: 0.2217
Epoch 6/10
237/237 [==============================] - 300s 1s/step - loss: 3.5725 - acc: 0.2567
Epoch 7/10
237/237 [==============================] - 329s 1s/step - loss: 3.2618 - acc: 0.2958
Epoch 8/10
237/237 [==============================] - 359s 2s/step - loss: 2.9882 - acc: 0.3341
Epoch 9/10
237/237 [==============================] - 246s 1s/step - loss: 2.7545 - acc: 0.3734
Epoch 10/10
237/237 [==============================] - 264s 1s/step - loss: 2.5555 - acc: 0.4067


1/1 [==============================] - 0s 57ms/step
One of the oldest and most expert among them obtained he was absolutely incredulous here she would not be insensible


In [20]:
import pickle
with open('model_pickle','wb') as f:
    pickle.dump(model,f)



Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\bidirectional
......vars
...layers\bidirectional\backward_layer
......vars
...layers\bidirectional\backward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\forward_layer
......vars
...layers\bidirectional\forward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\layer
......vars
...layers\bidirectional\layer\cell
......vars
...layers\dense
......vars
.........0
.........1
...layers\embedding
......vars
.........0
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Mo

In [ ]:
seed_text = "One of the oldest and most expert among them obtained"
next_words = 10

for _ in range(next_words):
    sequence = tokenizer.texts_to_sequences([seed_text])[0]
    print(sequence)
    padded = pad_sequences([sequence], maxlen=max_seq_len - 1)
    predicted = model.predict(padded)[0]
    predicted_word_index = np.argmax(predicted)
    output_word = ''
    
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            output_word = word
            break
    
    seed_text += ' ' + output_word
    
print(seed_text)

In [66]:
# # loss and accuracy analysis 

# import matplotlib.pyplot as plt

# history = model.history
# print(history.history.keys())

# acc = history.history['accuracy']
# loss = history.history['loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'b', label="Training accuracy")
# plt.title("Training accuracy")
# plt.figure()
# plt.plot(epochs, loss, 'b', label="Training loss")
# plt.title("Training loss")
# plt.legend()
# plt.show()

In [1]:
import pickle